In [1]:
from utils.config import load_datasets


def load_dataset(datasets):
    datasets = load_datasets(folder="datasets", names=datasets)
    return datasets

datasets = load_dataset(['dataDIVAT1_train'])
print(datasets['dataDIVAT1_train'].head())

       pid  event  time   num_ageR  num_year_tx fac_sexR fac_graft fac_year_tx
1417  1417      0   182  56.732559         2012   female       Yes        2012
3683  3683      0  1725  59.314423         2007   female        No        2007
4807  4807      0    85  49.619428         2007     male       Yes        2007
3994  3994      0     1  27.946008         2011     male        No        2011
3220  3220      1   518  26.861790         2000     male        No        2000


In [3]:
import torch
torch.cuda.is_available()

True

Survival Gan

In [6]:
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import SurvivalAnalysisDataLoader
import torch 


device = torch.device("cuda")
# https://github.com/vanderschaarlab/synthcity/issues/249
data = SurvivalAnalysisDataLoader(datasets['dataDIVAT1_train'], target_column="event", time_to_event_column="time")
model = Plugins().get("survival_gan", device=device)
model.fit(data)

  8%|▊         | 849/10000 [04:04<44:00,  3.47it/s]  


Survival VAE

In [13]:
from synthcity.plugins import Plugins
from synthcity.plugins.core.dataloader import SurvivalAnalysisDataLoader
import torch 


device = torch.device("cuda")

data = SurvivalAnalysisDataLoader(datasets['dataDIVAT1_train'], target_column="event", time_to_event_column="time")
model = Plugins().get("survae", device=device)
model.fit(data)


100%|██████████| 1000/1000 [03:10<00:00,  5.25it/s]


In [14]:
model.generate(1000)

,pid,event,time,num_ageR,num_year_tx,fac_sexR,fac_graft,fac_year_tx
0,656,0,1441,29.595931,2007,male,No,2007
1,4048,0,280,44.408547,2006,female,No,2006
2,2796,0,102,71.670634,2011,male,No,2011
3,5367,0,284,39.587495,2007,female,No,2007
4,1556,0,19,47.298226,2006,male,No,2006
...,...,...,...,...,...,...,...,...
995,5181,0,329,58.306481,2009,female,No,2009
996,2282,0,2879,65.358153,2001,female,Yes,2001
997,3356,0,416,56.848102,2004,male,No,2004
998,4227,0,789,24.972388,2007,female,No,2007
